In [ ]:
import os

""""
Devo code for all kinds of state space plotting, especially things like dPCA and tsne.

NOTE: previously dPCA plots:
- 240128_snippets_demixed_PCA
- analy_dpca_plot_script.py [More general purpose]
- analyquick_dpca_script_substrokes [Focusing on substrokes]
(Here, try to consolidate all of these things)

"""

# Load a prviously saved DFallPA
(This saving is only done by hand)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
1

In [ ]:

# this is the path to the dataset
# path = '/gorilla1/analyses/recordings/main/RSA/Diego-230615/agg_True-subtr_None-dist_euclidian_unbiased/SP_shape_loc/DFallpa.pkl'
# path = "/gorilla4/Dropbox/SCIENCE/FREIWALD_LAB/DATA/for_xuan/DFallpa.pkl"
# path = "/gorilla4/Dropbox/SCIENCE/FREIWALD_LAB/DATA/for_xuan/DFallpa_samp_and_stroke.pkl"
# path = "/gorilla4/Dropbox/SCIENCE/FREIWALD_LAB/DATA/for_xuan/DFallpa_pig_planning.pkl"
# path = "/gorilla4/Dropbox/SCIENCE/FREIWALD_LAB/DATA/for_xuan/DFallpa_pig_concat_trial_and_stroke_which_levels.pkl"
# path = "/gorilla4/Dropbox/SCIENCE/FREIWALD_LAB/DATA/for_xuan/DFallpa_char_concat_trial_and_stroke_Pancho_230126.pkl"
path = "/gorilla4/Dropbox/SCIENCE/FREIWALD_LAB/DATA/for_xuan/DFallpa_shape_loc_Pancho_220715.pkl"


In [ ]:
DFallpa = pd.read_pickle(path)


## Make sure to normalize PA before running any modeling on it:

In [ ]:
list_panorm = []
for pa in DFallpa["pa"].tolist():
    from neuralmonkey.analyses.state_space_good import popanal_preprocess_scalar_normalization
    PAnorm, PAscal, PAscalagg, fig, axes, groupdict = popanal_preprocess_scalar_normalization(pa, None, DO_AGG_TRIALS=False)
    list_panorm.append(PAnorm)
DFallpa["pa"] = list_panorm

# del DFallpa["pa_norm"]

# Generate a DFallpa (from Snippets)

In [ ]:
# Char, 
# animal = "Pancho"
# date = 230126
# do_combine = True

# Single prim, novels
animal = "Diego"
# date = 221220
date = 230630
do_combine = False

if do_combine:
    # COMBINE trial and stroke
    dir_suffix = "test"
    question = None
    # q_params = None
    which_level = None
    q_params = {
        "effect_vars": ["seqc_0_shape", "seqc_0_loc"]
    }
    
    combine_trial_and_stroke = True
    
    # PIG
    # question_trial = "PIG_BASE_trial"
    # question_stroke = "PIG_BASE_stroke"
    # check_that_locs_match = True
    
    # CHAR
    question_trial = "CHAR_BASE_trial"
    question_stroke = "CHAR_BASE_stroke"
    check_that_locs_match = True
    check_that_shapes_match = True
else:
    # DONT COMBINE, use questions.
    # question = "CHAR_BASE_trial"
    question = "PIG_BASE_trial"
    # question = "SP_novel_shape"
    # question = "SP_shape_loc"
    # question = "SP_shape_size"
    combine_trial_and_stroke = False
    which_level = "trial" # Doesnt matter
    dir_suffix = question

    # Load q_params
    from neuralmonkey.analyses.rsa import rsagood_questions_dict, rsagood_questions_params
    q_params = rsagood_questions_dict(animal, date, question)[question]

############### PARAMS
exclude_bad_areas = True
SPIKES_VERSION = "tdt" # since Snippets not yet extracted for ks
combine_into_larger_areas = False
HACK_RENAME_SHAPES = False
list_time_windows = [(-0.6, 0.6)]

########################################## RUN

if combine_trial_and_stroke:
    from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper_combine_trial_strokes
    DFallpa = dfallpa_extraction_load_wrapper_combine_trial_strokes(animal, date, question_trial,
                                                                       question_stroke,
                                                list_time_windows,
                                               combine_into_larger_areas = combine_into_larger_areas,
                                               exclude_bad_areas=exclude_bad_areas,
                                                SPIKES_VERSION="tdt",
                                                HACK_RENAME_SHAPES = HACK_RENAME_SHAPES,
                                               do_fr_normalization=True,
                                                    check_that_shapes_match=check_that_shapes_match,
                                                check_that_locs_match=check_that_locs_match)
else:
    from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper
    DFallpa = dfallpa_extraction_load_wrapper(animal, date, question, list_time_windows,
                                              which_level=which_level,
                                              combine_into_larger_areas = combine_into_larger_areas,
                                              exclude_bad_areas = exclude_bad_areas,
                                              SPIKES_VERSION = SPIKES_VERSION,
                                              HACK_RENAME_SHAPES = HACK_RENAME_SHAPES,
                                              do_fr_normalization=True)

##### To save DFallPA

In [ ]:
# Save it
import pickle
path = "/gorilla4/Dropbox/SCIENCE/FREIWALD_LAB/DATA/for_xuan/DFallpa_shape_loc_Pancho_220715.pkl"
with open(path, "wb") as f:
    pickle.dump(DFallpa, f)
print("Saved to:", path)


In [ ]:
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os
SAVEDIR_ANALYSIS = f"{PATH_ANALYSIS_OUTCOMES}/recordings/main/statespacegood"

# PREPROCESS - factorize all relevant labels FIRST here.


In [ ]:
from neuralmonkey.analyses.decode_good import preprocess_factorize_class_labels_ints
MAP_LABELS_TO_INT = preprocess_factorize_class_labels_ints(DFallpa)

# [TESTING] State space plots (tsne)
NOTE: previously have dPCA plots...

### Preprocess PA

In [ ]:
import numpy as np
# Extract information about the first stroke (semantic labels)
locations_allpa =[]
angles_allpa =[]
list_i_j = []
for i, pa in enumerate(DFallpa["pa"]):
    dflab = pa.Xlabels["trials"]
    
    list_seqc_0_shsem = []
    list_seqc_0_locon = []
    for j, row in dflab.iterrows():
        
        # To get semantic label for shape, use task token
        Tk = row["Tkbeh_stktask"]
        Tk.features_extract_wrapper(features_get=["shape_semantic"])
        list_seqc_0_shsem.append(Tk.Tokens[0]["shape_semantic"])
        
        # To get motoro stuff
        Tk = row["Tkbeh_stkbeh"]
        Tk.features_extract_wrapper(features_get=["loc_on", "angle_on"])
        list_seqc_0_locon.append(Tk.Tokens[0]["loc_on"])

        # Collect acorss pa
        locations_allpa.append(Tk.Tokens[0]["loc_on"])
        angles_allpa.append(Tk.Tokens[0]["angle_on"])
        list_i_j.append((i,j))
        
    dflab["seqc_0_shsem"] = list_seqc_0_shsem
    dflab["seqc_0_locon"] = list_seqc_0_locon
    dflab["seqc_0_locx"] = np.array(list_seqc_0_locon)[:,0]
    dflab["seqc_0_locy"] = np.array(list_seqc_0_locon)[:,1]
    
    
# BIN LOCATIONS
from pythonlib.tools.nptools import bin_values_by_rank
# Given list of locations, bin them in x and y
import numpy as np
tmp = np.stack(locations_allpa)
xs = tmp[:,0]
ys = tmp[:,1]

xs_binned = bin_values_by_rank(xs, nbins=2)
ys_binned = bin_values_by_rank(ys, nbins=2)
locations_allpa_binned = np.stack([xs_binned, ys_binned], axis=1)
# map from (i,j) to locations_binned
map_ij_to_locbinned ={}
assert len(list_i_j)==len(locations_allpa_binned)
for (i,j), loc_binned in zip(list_i_j, locations_allpa_binned):
    map_ij_to_locbinned[(i,j)] = loc_binned
# Place back into pa
colname = "seqc_0_loconbinned"
for i, pa in enumerate(DFallpa["pa"]):
    dflab = pa.Xlabels["trials"]
    vals = []
    for j, row in dflab.iterrows():
        vals.append(tuple(map_ij_to_locbinned[(i,j)].tolist()))
    dflab[colname] = vals 

# BIN ANGLES
from pythonlib.tools.vectools import bin_angle_by_direction
angles_binned = bin_angle_by_direction(angles_allpa, num_angle_bins=4)

map_ij_to_anglebinned ={}
map_ij_to_angle ={}
assert len(list_i_j)==len(angles_binned)
for (i,j), ab, a in zip(list_i_j, angles_binned, angles_allpa):
    map_ij_to_anglebinned[(i,j)] = ab
    map_ij_to_angle[(i,j)] = a
    
# Place back into pa
colname = "seqc_0_anglebinned"
for i, pa in enumerate(DFallpa["pa"]):
    dflab = pa.Xlabels["trials"]
    vals = []
    for j, row in dflab.iterrows():
        vals.append(map_ij_to_anglebinned[(i,j)])
    dflab[colname] = vals 

colname = "seqc_0_angle"
for i, pa in enumerate(DFallpa["pa"]):
    dflab = pa.Xlabels["trials"]
    vals = []
    for j, row in dflab.iterrows():
        vals.append(map_ij_to_angle[(i,j)])
    dflab[colname] = vals 


In [ ]:
# Get angle as continuous variable, not just binned
    

In [ ]:
dflab.loc[:, ["seqc_0_locon", "seqc_0_loconbinned", "seqc_0_anglebinned"]]


In [ ]:
dflab["seqc_0_loconbinned"].value_counts()

### Helper functions for tsne

In [ ]:
from neuralmonkey.analyses.state_space_good import dimredgood_nonlinear_embed_data

In [ ]:
import umap.plot
umap.plot.points(mapper, labels=dflab["seqc_0_shape"], theme="fire")
import umap.plot
umap.plot.points(mapper, labels=dflab["seqc_0_shape"], theme="fire")
umap.plot.connectivity(mapper, show_points=True, edge_bundling='hammer')
umap.plot.diagnostic(mapper, diagnostic_type='pca')

# p = umap.plot.interactive(mapper, labels=dflab["seqc_0_shape"], hover_data=hover_data, point_size=2)
p = umap.plot.interactive(mapper, labels=dflab["seqc_0_shape"], point_size=2)
umap.plot.show(p)

### Extract data

In [ ]:
from neuralmonkey.analyses.state_space_good import trajgood_construct_df_from_raw, trajgood_plot_colorby_splotby, trajgood_plot_colorby_splotby_scalar
from pythonlib.tools.pandastools import append_col_with_grp_index
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os

### PARAMS
tbin_dur = 0.1
tbin_slide = 0.1
reshape_method = "ntrials_x_nchans_ntimes"
animal = "Pancho"
date = 220715
SAVEDIR_ANALYSIS = f"{PATH_ANALYSIS_OUTCOMES}/recordings/main/STATE_SPACE_GENERAL/{animal}-{date}"
RES = []

for i, row in DFallpa.iterrows():
    pa = row["pa"]
    br = row["bregion"]
    wl = row["which_level"]
    ev = row["event"]
    tw = row["twind"]

    # print(br, wl, ev, tw)
    # # if br=="vlPFC_p" and ev=="03_samp":
    # # if br=="PMd_a" and ev=="03_samp":
    # # if br=="M1_m" and ev=="06_on_strokeidx_0":
    # if br=="PMv_m" and ev=="03_samp":
    # # if br=="PMv_m" and ev=="06_on_strokeidx_0":
    #     print("GOOD")
    #     break
    
    if ev=="03_samp":
        list_twind_overall = [
            [0.2, 0.6]
        ]
    elif ev=="06_on_strokeidx_0":
        list_twind_overall = [
            [-0.4, 0],
            [0, 0.4]
        ]
    else:
        print(ev)
        assert False
        
    # if br=="PMv_m" and ev=="06_on_strokeidx_0":
    for twind_overall in list_twind_overall:
                
        # Extract data
        X, pathis = pa.dataextract_state_space_decode_flex(twind_overall, tbin_dur, tbin_slide, reshape_method)
        dflab = pathis.Xlabels["trials"]
        dflab = append_col_with_grp_index(dflab, ["seqc_0_shape", "seqc_0_loc"], "seqc_0_shapeloc")

        STROKES_BEH = [tk.Tokens[0]["Prim"].Stroke() for tk in dflab["Tkbeh_stkbeh"]]
        STROKES_TASK = [tk.Tokens[0]["Prim"].Stroke() for tk in dflab["Tkbeh_stktask"]] 
        
        # Embed data
        for METHOD in ["umap", "tsne"]:
            Xredu = dimredgood_nonlinear_embed_data(X, METHOD=METHOD, n_components=2, tsne_perp="auto", umap_n_neighbors="auto")
                
            ##### Plot scalars
            savedir = f"{SAVEDIR_ANALYSIS}/{br}-{ev}-twind={'_'.join([str(t) for t in twind_overall])}-METHOD={METHOD}"
            print(savedir)
            os.makedirs(savedir, exist_ok=True)
            
            # List of plots to make
            list_var_color_var_subplot = [
                ["seqc_0_shape", "seqc_0_loc"],
                ["seqc_0_loc", "seqc_0_shape"],
                ["seqc_0_locx", "seqc_0_shape"],
                ["seqc_0_locy", "seqc_0_shape"],
                ["seqc_0_angle", "seqc_0_shape"],
                ["seqc_0_angle", "seqc_0_loc"],
                ["seqc_0_shape", "seqc_0_anglebinned"],
                ["seqc_0_shape", "shape_is_novel_all"]]
            
            if False:
                ##### Create meaned data to plot
                effect_vars = ["seqc_0_shape", "seqc_0_loc", "shape_is_novel_all", "seqc_0_shsem", "seqc_0_loconbinned", "seqc_0_anglebinned"]
                pathis_meaned = pathis.slice_and_agg_wrapper("trials", effect_vars)
            

            for var_color, var_subplot in list_var_color_var_subplot:
                xs = Xredu[:,0]
                ys = Xredu[:,1]
                labels_color = dflab[var_color].tolist()
                # text_to_plot = labels_color
                text_to_plot = None
                labels_subplot = dflab[var_subplot].tolist()
                fig, axes, map_levo_to_ax, map_levo_to_inds = trajgood_plot_colorby_splotby_scalar(xs, ys, labels_color, labels_subplot, var_color, 
                                                                       var_subplot, SIZE=7,
                                                                       overlay_mean=True, text_to_plot=text_to_plot,
                                                                       STROKES_BEH=STROKES_BEH, STROKES_TASK=STROKES_TASK)
                savefig(fig, f"{savedir}/color={var_color}-sub={var_subplot}.pdf")
                plt.close("all")
            assert False

In [ ]:
# TODO:
# - collect all --> 1 figure for each bregion, etc.
# - Other datasets:
# - novel prims
# ---- sim structure
# ---- novel cluster together?
# - shape vs. size
# - PIG
# - char

###### Plot overlaying stroke drawing associated with each shape [DONE, moved above]

In [ ]:
# Extract strokes
# from pythonlib.tools.stroketools import strokes_alignonset, strokes_centerize
import matplotlib.pyplot as plt
# strokes_ver = "beh"
STROKES_BEH = [tk.Tokens[0]["Prim"].Stroke() for tk in dflab["Tkbeh_stkbeh"]]
STROKES_TASK = [tk.Tokens[0]["Prim"].Stroke() for tk in dflab["Tkbeh_stktask"]] 
    

In [ ]:
xs = Xredu[:,0]
ys = Xredu[:,1]
labels_color = dflab[var_color].tolist()
labels_color = np.random.rand(len(xs))
# text_to_plot = labels_color
text_to_plot = None
labels_subplot = dflab[var_subplot].tolist()

fig, axes, map_levo_to_ax, map_levo_to_inds = trajgood_plot_colorby_splotby_scalar(xs, ys, labels_color, labels_subplot, var_color, var_subplot, SIZE=7,
                                                       overlay_mean=True, text_to_plot=text_to_plot,
                                                                                          STROKES_BEH=STROKES_BEH, STROKES_TASK=STROKES_TASK)

# from pythonlib.tools.pandastools import grouping_append_and_return_inner_items
# grpdict = grouping_append_and_return_inner_items(dflab, [var_color, var_subplot])
# levels_var_color = dflab[var_color].unique().tolist()
# 
# for strokes, align_to, pcol in [
#     [STROKES_BEH, "onset", "k"],
#     [STROKES_TASK, "center", "m"]
# ]:
#     if False:
#         # Overlay nplot random strokes
#         ax = axes.flatten()[0]
#         nplot = 40
#         overlay_stroke_on_plot_mult_rand(strokes, xs, ys, nplot, align_to)
#     
#     # Overlay random strokes, sampling within each level of labels_color
#     nplot = 4
#     for levo, ax in map_levo_to_ax.items():
#         for lev in levels_var_color:
#             inds = grpdict[lev, levo]
#             overlay_stroke_on_plot_mult_rand([strokes[i] for i in inds], xs[inds], ys[inds], nplot, align_to, color=pcol)
        

In [ ]:

def overlay_stroke_on_plot_mult_rand(strokes, xs, ys, nrand=10, align_to="onset", color=None):
    import random

    inds = range(len(strokes))

    if len(inds)>nrand:
        inds = random.sample(inds, nrand)

    for i in inds:
        strok = strokes[i][:, :2]
        # times = strokes[i][:, 2]
        x = xs[i]
        y = ys[i]
        overlay_stroke_on_plot(strok, x, y, ax, align_to=align_to, color=color)
        

def overlay_stroke_on_plot(strok, x, y, ax, SIZE = 0.05, align_to="onset", color=None):
    """
    PARAMS:
    - SIZE, frac of screen to make the size of the strok.
    """
    
    if color is None:
        color="k"
    from pythonlib.tools.stroketools import strokes_bounding_box_dimensions
    # Get dimensions of figure, so can correctly scale stroke
    XLIM = ax.get_xlim()
    YLIM = ax.get_ylim()
    xdiff = XLIM[1] - XLIM[0]
    ydiff = YLIM[1] - YLIM[0]
    d_ax = (xdiff**2 + ydiff**2)**0.5 
        
    # Decide if align at onset (beh) or center (task)
    from pythonlib.tools.stroketools import strokes_alignonset, strokes_centerize
    import matplotlib.pyplot as plt
    if align_to == "onset":
        strok = strokes_alignonset([strok])[0]
    elif align_to == "center":
        strok = strokes_centerize([strok])[0]
    elif align_to is None:
        pass
    else:
        print(align_to)
        assert False
            
    # Rescale stroke
    w, h, d = strokes_bounding_box_dimensions([strok])
    rescale = SIZE * d_ax/d 
    aspect = [1, ydiff/xdiff] 
    strok = strok * rescale # rescale to page
    strok = strok * aspect # to make sure strok aspect is visually accurate
    strok = strok + [x,y] # move to location
    
    # Plot
    ax.plot(strok[:,0], strok[:,1], '-', alpha=0.7, color=color)
    # ax.scatter(strok[:,0], strok[:,1], c=times)
    # if align_to=="onset":
    ax.plot(x, y, 'o', mfc="none", alpha=0.4, color=color)
        
    


In [ ]:
dflab["Tkbeh_stktask"].values[0].Tokens[0]["Prim"].Stroke()

In [ ]:
# load DS for this animal on this day
 # Load a daily dataset
from pythonlib.dataset.dataset import load_dataset_notdaily_helper, load_dataset_daily_helper

D = load_dataset_daily_helper(animal, date)

In [ ]:
from pythonlib.dataset.dataset_strokes import preprocess_dataset_to_datstrokes, DatStrokes
# DS = DatStrokes(D)
DS = preprocess_dataset_to_datstrokes(D, "all_no_abort_superv")

In [ ]:
best_n_by_dist = 15
DS.distgood_compute_beh_task_strok_distances()
dfdat = DS.features_generate_dataset_mean_primshape("shape", best_n_by_dist=best_n_by_dist)


In [ ]:
shape = "novelprim--6288514134535433462-90-3.4-0.9"
inds = DS.find_indices_this_shape(shape)
titles = [DS.stroke_shape_align_flip_vs_task(i) for i in inds] 
DS.plot_multiple_overlay_entire_trial(inds, overlay_beh_or_task="task", titles=titles);


In [ ]:
# Given a shape, extract strokes and task
shape = "circle-67-6.0-0.4"
n = 5
inds = DS.find_indices_this_shape(shape)
# pick n random inds
import random
inds = random.sample(inds, n)

strokes_beh = DS.extract_strokes(inds=inds, ver_behtask="beh")
strokes_task = DS.extract_strokes(inds=inds, ver_behtask="task_aligned_single_strok")

# for all beh, align to stroke onset
from pythonlib.tools.stroketools import strokes_alignonset
import matplotlib.pyplot as plt
strokes_beh = strokes_alignonset(strokes_beh)

fig, ax = plt.subplots()

XLIM = ax.get_xlim()
YLIM = ax.get_ylim()
xdiff = XLIM[1] - XLIM[0]
ydiff = YLIM[1] - YLIM[0]
strokes_beh = [[x, y] + strok[:,:2] * xdiff/ydiff for strok in strokes_beh]
# strok[:,0] =  strok[:,0] * xdiff/ydiff # so it looks correct even if aspect is not 1
# strok = strok + loc # move to new locatin

# plot at that location, starting from strok onset
for strok in strokes_beh:
    ax.plot(strok[:,0], strok[:,1], '-')
    ax.plot(strok[0,0], strok[0,1], 'ok')



In [ ]:
from pythonlib.drawmodel.strokePlots import plotDatStrokes
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
plotDatStrokes([strok], ax)

In [ ]:
# var_color = "seqc_0_shape"
var_color = "seqc_0_anglebinned"
# var_color = "seqc_0_loconbinned"
var_subplot = "shape_is_novel_all"

xs = Xtsne[:,0]
ys = Xtsne[:,1]
labels_color = dflab[var_color].tolist()
labels_subplot = dflab[var_subplot].tolist()
fig, axes= trajgood_plot_colorby_splotby_scalar(xs, ys, labels_color, labels_subplot, var_color, var_subplot, SIZE=7,
                                                       overlay_mean=False, text_to_plot=labels_color,
                                                       strokes_to_overlay=strokes_to_overlay)

In [ ]:
dflab["seqc_0_shape"]

In [ ]:
shapes = dflab["shap"]

In [ ]:
DS.Version

In [ ]:


pa.Xlabels["trials"].keys()

In [ ]:
xs = Xtsne[:,0]
ys = Xtsne[:,1]
labels_color = dflab[var_color].tolist()
labels_subplot = dflab[var_subplot].tolist()
fig, axes= trajgood_plot_colorby_splotby_scalar(xs, ys, labels_color, labels_subplot, var_color, var_subplot, SIZE=4);

In [ ]:
# Plot, subsampling to N trials for each level

##### Plot trajectories

In [ ]:


# INPUT PARAMS
var_color_by = "seqc_0_shape"
var_subplots = "seqc_0_loc"

# Runs
# 1) Construct dataframe
# df = pa.dataextract_split_by_label_grp_for_statespace(grpvars)
grpvars = [var_color_by, var_subplots]
labels = pa.Xlabels["trials"].loc[:, grpvars] 
labelvars = grpvars
df = trajgood_construct_df_from_raw(X, pa.Times, labels, labelvars)

# 2) Plot
dims = (2,3) # pairs of dimensions to plot
times_to_mark = [0.] # you can mark specific times on the plot. here marks the 0. sec mark.
times_to_mark_markers = ["d"] # mark with a diamond ("d")
time_bin_size = 0.05 # to make plot easier to visaulize, you can bin in time.
trajgood_plot_colorby_splotby(df, var_color_by, var_subplots, dims, "traj", mean_over_trials=True,
                              times_to_mark = times_to_mark, times_to_mark_markers = times_to_mark_markers,
                              time_bin_size=time_bin_size)

In [ ]:
bregion = "PMv_m"
twind = (-0.6, 0.6)
event = "03_samp"


In [ ]:
DFallpa